#### Some setup to connect container paths to the notebook (required each time notebook is run)

In [0]:
tiers = ["bronze","silver","gold"]
adl_paths = {tier : f'abfss://{tier}@myfirststorageaccount271.dfs.core.windows.net/' for tier in tiers}

bronze_adls = adl_paths['bronze']
silver_adls = adl_paths['silver']
gold_adls = adl_paths['gold']

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)


[]

### Request and Save Earthquake info

In [0]:
import requests
import json
from datetime import datetime
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try: 
    response = requests.get(url)
    response.raise_for_status()
    data = response.json().get('features',[])

    if not data:
        print('No features available for the specific range')

    else:
        file_path = f"{adl_paths['bronze']}/{start_date}_earthquake_data.json"
        json_data = json.dumps(data, indent = 4)
        dbutils.fs.put(file_path,json_data, overwrite = True)

        print(f"Data written to {file_path}")

except requests.exceptions.RequestException as e :
    print(f'Error fetching data from the Earthquake API : {e}')

Wrote 301272 bytes.
Data written to abfss://bronze@myfirststorageaccount271.dfs.core.windows.net//2025-05-17_earthquake_data.json


#### Send information 

In [0]:
output_data = {
    'bronze_adls' : bronze_adls,
    'silver_adls' : silver_adls,
    'gold_adls' : gold_adls,
    'start_date' : start_date.isoformat(),
    'end_date' : end_date.isoformat()
}
dbutils.jobs.taskValues.set(key = "bronze_output",value = output_data)

{'type': 'Feature',
 'properties': {'mag': 1.6,
  'place': '62 km W of Happy Valley, Alaska',
  'time': 1747526354601,
  'updated': 1747526522582,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak0256ayo6y7',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak0256ayo6y7&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 39,
  'net': 'ak',
  'code': '0256ayo6y7',
  'ids': ',ak0256ayo6y7,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.83,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - 62 km W of Happy Valley, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-152.8335, 60.0448, 95.2]},
 'id': 'ak0256ayo6y7'}